
#### 介绍

In [1]:
'''
输入待爬取的内容和爬取的页数，即可自动开始爬取视频。视频来自b站。
'''

'\n输入待爬取的内容和爬取的页数，即可自动开始爬取视频。视频来自b站。\n'

#### 导入模块

In [2]:
import requests
from lxml import etree
import time,os,re
from tqdm import tqdm

#### 功能函数

In [3]:
class get_video:
    def __init__(self):
        # 伪装成chrome浏览器进行网页登录
        self.headers  = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36'}
        # 文件保存地址
        self.dirs_save = './爬取的文件/'
     
    # 输入网址，获得该网址的解析
    def get_selector(self,url):
        response = requests.get(url,headers=self.headers)
        html_data = response.text
        # 数据为html文件，将html转换成py文件
        selector = etree.HTML(html_data)
        return selector
        
    def check_name(self,s_name):
        """处理文件名非法字符的方法"""
        s_name = s_name.replace(' ','-')
        pattern = re.compile(r"[\/\\\:\*\?\"\<\>\|]")  # '/ \ : * ? " < > |'
        new_name = re.sub(pattern, "_", s_name)  # 替换为下划线
        return new_name

    def download_file(self,url, file_name, headers):
        r = requests.get(url,headers=headers, stream=True)
        chunk_size = 1024*1024
        content_size = int(r.headers['content-length'])/1024/1024  
        print('开始下载...大小为：{:.2f}MB'.format(content_size))
        f = open(file_name, "wb")
        for chunk in tqdm(r.iter_content(chunk_size=chunk_size)):
            f.write(chunk)
        print('{}文件保存完毕...'.format(file_name))
        return content_size

#### ·输入待爬取的内容和页数

In [4]:
# 输入要爬取的内容
#content = input('请输入要爬取的内容:')
content = '搞笑短视频'
# 输入爬取的页数
#pages = int(input('请输入爬取的页数：'))
pages = 1


#### ·开始爬取

In [ ]:
crawler = get_video()
if not os.path.exists(crawler.dirs_save):
    os.makedirs(crawler.dirs_save)

# 确认目标的url,设置if条件，进行判断
for i in range(pages):
    #url_page = get_url_page(i)
    url = f'https://search.bilibili.com/all?keyword={content}&from_source=nav_search&spm_id_from=333.851.b_696e7465726e6174696f6e616c486561646572.17&page={i+1}'
    # 获得该网页的解析
    selector = crawler.get_selector(url)
    # 提取视频页面对应的url列表
    url_list = selector.xpath('//li[@class="video-item matrix"]/a[@target="_blank"]/@href')    
    
    for url in url_list:
        # 由于url并不是完整的，因此需要拼接
        url_1 = 'https:' + url 
        # 发送请求，获取响应
        print('______________________')
        print('网页数据处理中...')
        # 获得该网页的解析
        selector = crawler.get_selector(url_1)
        # 提取视频的名称
        title_ = selector.xpath('//title/text()')[0]
        # 对名称正则处理
        title_ = re.findall(r'(.*?)_哔哩哔哩',title_)[0]
        title_ = title_.replace('/','')
        # 提取纯视频和纯音频的url
        url_str = selector.xpath('//script[contains(text(),"window.__playinfo__")]/text()')[0]
        video_url = re.findall('"video":\[{"id":\d+,"baseUrl":"(.*?)",',url_str)[0]
        audio_url = re.findall('"audio":\[{"id":\d+,"baseUrl":"(.*?)",',url_str)[0]

        headers_ = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
            'Referer':url_1
        }

        print('下载临时视频文件')
        video_name = 'tem.mp4'
        crawler.download_file(video_url, video_name, headers_)

        print('下载临时音频文件')
        audio_name = 'tem.mp3'
        crawler.download_file(audio_url, audio_name, headers_)

        video_save = crawler.dirs_save + crawler.check_name(f'{title_}.mp4')
        print('cmd执行命令：',f'ffmpeg -i {audio_name} -i {video_name} -c copy  {video_save} -loglevel quiet')
        os.system(f'ffmpeg -i {audio_name} -i {video_name} -c copy  {video_save} -loglevel quiet')
        # 等待视频合成
        while False == os.path.exists(video_save):
            print('视频合成中')
            time.sleep(1)
        # 打印合成后视频的大小
        res_size = int(os.stat(video_save).st_size/1024)
        res_size1 = int(res_size/1024)
        print(f'{video_save}视频文件合成后的大小为：{res_size}KB,{res_size1}MB')

        # 将纯视频和纯音频文件删除
        os.remove(audio_name)
        os.remove(video_name)
        print('______________________')

______________________
网页数据处理中...
下载临时视频文件


0it [00:00, ?it/s]

开始下载...大小为：7.05MB


8it [00:01,  4.40it/s]


tem.mp4文件保存完毕...
下载临时音频文件


...

tem.mp3文件保存完毕...
cmd执行命令： ffmpeg -i tem.mp3 -i tem.mp4 -c copy  ./爬取的文件/国外逗比短视频#15，嘟嘟嘴黑人Lucas的合集.mp4 -loglevel quiet
./爬取的文件/国外逗比短视频#15，嘟嘟嘴黑人Lucas的合集.mp4视频文件合成后的大小为：38447KB,37MB
______________________
______________________
网页数据处理中...


0it [00:00, ?it/s]

下载临时视频文件
开始下载...大小为：32.48MB


33it [00:05,  5.56it/s]


tem.mp4文件保存完毕...
下载临时音频文件
